In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import pandas as pd

train_file_path = '/content/drive/My Drive/small-PhoMT/small-train.json'
dev_file_path = '/content/drive/My Drive/small-PhoMT/small-dev.json'
test_file_path = '/content/drive/My Drive/small-PhoMT/small-test.json'

# Mở và đọc file JSON
with open(train_file_path, 'r', encoding='utf-8') as f:
    train_data = pd.read_json(f)

with open(dev_file_path, 'r', encoding='utf-8') as f:
    dev_data = pd.read_json(f)

with open(test_file_path, 'r', encoding='utf-8') as f:
    test_data = pd.read_json(f)

# Bây giờ bạn có thể bắt đầu tiền xử lý hoặc huấn luyện mô hình
print(f"Số lượng bản ghi trong train data: {len(train_data)}")
print(f"Số lượng bản ghi trong valid data: {len(dev_data)}")
print(f"Số lượng bản ghi trong test data: {len(test_data)}")

Số lượng bản ghi trong train data: 20000
Số lượng bản ghi trong valid data: 2000
Số lượng bản ghi trong test data: 2000


In [3]:
!pip install torch==2.3.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 102.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 122.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1

In [4]:
!pip install torchtext==0.18.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.5 MB/s eta 0:00:00


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from collections import Counter
import pandas as pd
from transformers import AutoTokenizer
import numpy as np
import random
import math
from tqdm.notebook import tqdm
import time
from tqdm.notebook import tqdm

# Đặt seed để đảm bảo kết quả có thể lặp lại
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

/usr/local/lib/python3.12/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [6]:
tokenizer_vi = AutoTokenizer.from_pretrained('vinai/phobert-base', use_fast=False)

# Sử dụng Tokenizer cơ bản của Hugging Face cho tiếng Anh (Rất ổn định)
tokenizer_en = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=False)

# Sửa đổi quan trọng: Thêm thuộc tính clean-up
def clean_tokenizer_output(tokens, tokenizer):
    """
    Loại bỏ các token đặc biệt của tokenizer (như [CLS], [SEP], <s>, </s>)
    ra khỏi chuỗi output.
    """
    special_tokens = tokenizer.all_special_tokens
    # Lọc các token đặc biệt ra khỏi output
    return [t for t in tokens if t not in special_tokens]

def tokenize_en(text):
    """Tokenizes tiếng Anh bằng BERT Tokenizer và loại bỏ token đặc biệt."""
    tokens = tokenizer_en.tokenize(text.lower())
    return clean_tokenizer_output(tokens, tokenizer_en) # Lọc token đặc biệt của BERT

def tokenize_vi(text):
    """Tokenizes tiếng Việt bằng PhoBERT Tokenizer và loại bỏ token đặc biệt."""
    tokens = tokenizer_vi.tokenize(text.lower())

    # Lọc token đặc biệt của PhoBERT (<s>, </s>)
    cleaned_tokens = clean_tokenizer_output(tokens, tokenizer_vi)

    # Thêm token <sos> và <eos> đã được định nghĩa trong special_symbols của bạn
    return ['<sos>'] + cleaned_tokens + ['<eos>']

# Xây dựng Vocab
def yield_tokens(data_iter, language):
    """Hàm yield tokens từ DataFrame"""
    for _, row in data_iter.iterrows():
        if language == 'en':
            yield tokenize_en(row['english'])
        elif language == 'vi':
            yield tokenize_vi(row['vietnamese'])

# Các token đặc biệt
UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']

# Xây dựng Vocab
vocab_src = build_vocab_from_iterator(
    yield_tokens(train_data, 'en'),
    min_freq=2, # Chỉ giữ lại các từ xuất hiện ít nhất 2 lần
    specials=special_symbols
)
vocab_trg = build_vocab_from_iterator(
    yield_tokens(train_data, 'vi'),
    min_freq=2,
    specials=special_symbols
)

# Tự động gán index UNK_IDX cho những từ nằm ngoài vocab.
vocab_src.set_default_index(UNK_IDX)
vocab_trg.set_default_index(UNK_IDX)

# Kích thước từ vựng
INPUT_DIM = len(vocab_src)
OUTPUT_DIM = len(vocab_trg)
print(f"Kích thước từ vựng tiếng Anh (Input): {INPUT_DIM}")
print(f"Kích thước từ vựng tiếng Việt (Output): {OUTPUT_DIM}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Kích thước từ vựng tiếng Anh (Input): 10699
Kích thước từ vựng tiếng Việt (Output): 4699


In [7]:
# Biến dữ liệu DataFrame thành một bộ sưu tập các cặp Tensor đã được chỉ số hóa, sẵn sàng được nạp vào mô hình.
class TranslationDataset(Dataset):
    def __init__(self, df, vocab_src, vocab_trg):
        self.df = df
        self.vocab_src = vocab_src
        self.vocab_trg = vocab_trg

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        en_text = self.df.iloc[idx]['english']
        vi_text = self.df.iloc[idx]['vietnamese']

        # Tách token
        en_tokens = tokenize_en(en_text)
        vi_tokens = tokenize_vi(vi_text)

        # Chuyển token sang index
        en_indices = self.vocab_src.lookup_indices(en_tokens)
        vi_indices = self.vocab_trg.lookup_indices(vi_tokens)

        return torch.LongTensor(en_indices), torch.LongTensor(vi_indices)

# Collate function để padding các chuỗi trong cùng một batch
def collate_fn(batch):
    src_batch, trg_batch = [], []
    for src, trg in batch:
        src_batch.append(src)
        trg_batch.append(trg)

    # Padding: Hàm pad_sequence sẽ mặc định padding đến độ dài lớn nhất của sample trong batch
    src_batch = nn.utils.rnn.pad_sequence(src_batch, padding_value=PAD_IDX)
    trg_batch = nn.utils.rnn.pad_sequence(trg_batch, padding_value=PAD_IDX)

    return src_batch, trg_batch

# Tạo Dataset và DataLoader
BATCH_SIZE = 64
train_dataset = TranslationDataset(train_data, vocab_src, vocab_trg)
dev_dataset = TranslationDataset(dev_data, vocab_src, vocab_trg)
test_dataset = TranslationDataset(test_data, vocab_src, vocab_trg)

train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
dev_iterator = DataLoader(dev_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [8]:
class LuongAttention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        # Sử dụng phương pháp 'general' score: h_t^T * W * h_s
        # enc_hid_dim * 2 vì Encoder là Bidirectional
        self.attn = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

    def forward(self, decoder_hidden, encoder_outputs):
        # decoder_hidden = [batch size, dec hid dim] (h_t)
        # encoder_outputs = [src len, batch size, enc hid dim * 2] (h_s)

        src_len = encoder_outputs.shape[0]

        # 1. Chiếu encoder_outputs qua lớp Linear (W * h_s)
        # encoder_outputs_projected = [src len, batch size, dec hid dim]
        encoder_outputs_projected = self.attn(encoder_outputs)

        # 2. Chuẩn bị decoder_hidden để nhân ma trận
        # decoder_hidden = [batch size, 1, dec hid dim]
        decoder_hidden = decoder_hidden.unsqueeze(1)

        # 3. Đổi chiều encoder outputs để nhân
        # encoder_outputs_projected = [batch size, dec hid dim, src len]
        encoder_outputs_projected = encoder_outputs_projected.permute(1, 2, 0)

        # 4. Tính điểm Score bằng phép nhân ma trận (Batch Matrix Multiplication)
        # score = h_t * (W * h_s)
        # Output = [batch size, 1, src len]
        attention_scores = torch.bmm(decoder_hidden, encoder_outputs_projected)

        # 5. Softmax để ra trọng số
        # Output = [batch size, src len]
        return F.softmax(attention_scores.squeeze(1), dim=1)

In [9]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, enc_hid_dim, num_layers=3, bidirectional=True, dropout=dropout)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)

        # Xử lý hidden state để khởi tạo cho Decoder (như Bài 2)
        hidden_last = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden_last

In [10]:
class LuongDecoderInputFeeding(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)

        # --- THAY ĐỔI 1: INPUT-FEEDING ---
        # Input của LSTM = Embedding + Attentional Vector của bước trước (\tilde{h}_{t-1})
        # Kích thước input tăng lên: emb_dim + dec_hid_dim
        self.rnn = nn.LSTM(emb_dim + dec_hid_dim, dec_hid_dim, num_layers=3, dropout=dropout)

        self.wc = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.fc_out = nn.Linear(dec_hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs, cell, last_attentional_hidden):
        # input: [batch size]
        # last_attentional_hidden: [batch size, dec_hid_dim] (\tilde{h}_{t-1})

        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input)) # [1, batch, emb]

        # --- THAY ĐỔI 2: NỐI INPUT ---
        # input_feeding = [1, batch, dec_hid]
        last_attentional_hidden = last_attentional_hidden.unsqueeze(0)

        # rnn_input = [1, batch, emb + dec_hid]
        rnn_input = torch.cat((embedded, last_attentional_hidden), dim=2)

        # Chạy RNN
        output_rnn, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))

        # Tính Attention (như cũ)
        a = self.attention(hidden[-1], encoder_outputs)
        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted = torch.bmm(a, encoder_outputs)
        weighted = weighted.permute(1, 0, 2)

        # Tính Attentional Vector mới (\tilde{h}_t)
        rnn_output_sq = output_rnn.squeeze(0)
        weighted_sq = weighted.squeeze(0)

        concat_vector = torch.cat((weighted_sq, rnn_output_sq), dim=1)
        attentional_hidden = torch.tanh(self.wc(concat_vector))

        prediction = self.fc_out(attentional_hidden)

        # Trả về attentional_hidden mới để dùng cho bước sau
        return prediction, hidden, cell, attentional_hidden

In [11]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
class Seq2SeqInputFeeding(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)

        hidden = hidden.unsqueeze(0).repeat(3, 1, 1)
        cell = torch.zeros(3, batch_size, self.decoder.rnn.hidden_size).to(self.device)

        # --- THAY ĐỔI: KHỞI TẠO \tilde{h}_0 ---
        attentional_hidden = torch.zeros(batch_size, self.decoder.rnn.hidden_size).to(self.device)

        input = trg[0,:]

        for t in range(1, trg_len):
            # Truyền thêm attentional_hidden vào decoder
            output, hidden, cell, attentional_hidden = self.decoder(input, hidden, encoder_outputs, cell, attentional_hidden)

            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs

In [15]:
# --- CẤU HÌNH ---
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 256
DEC_HID_DIM = 256
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

# --- KHỞI TẠO MÔ HÌNH (SỬA ĐỔI QUAN TRỌNG) ---
# 1. Khởi tạo Attention (Vẫn dùng LuongAttention như cũ)
attn = LuongAttention(ENC_HID_DIM, DEC_HID_DIM)

# 2. Khởi tạo Encoder (Vẫn dùng Encoder cũ)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

# 3. Khởi tạo Decoder MỚI (LuongDecoderInputFeeding)
# Lưu ý: Phải dùng class LuongDecoderInputFeeding mà bạn vừa định nghĩa
dec = LuongDecoderInputFeeding(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

# 4. Khởi tạo Seq2Seq MỚI (Seq2SeqInputFeeding)
# Lưu ý: Phải dùng class Seq2SeqInputFeeding để xử lý luồng input-feeding
model = Seq2SeqInputFeeding(enc, dec, DEVICE).to(DEVICE)

# --- PHẦN CÒN LẠI GIỮ NGUYÊN ---
# Khởi tạo trọng số [-0.1, 0.1] (Rất quan trọng cho Luong Attention)
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.uniform_(param.data, -0.1, 0.1)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

# Optimizer & Loss
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

print(f'Số lượng tham số: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}')

# --- HÀM TRAIN & EVALUATE ---
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    # THÊM tqdm VÀO ĐÂY ĐỂ HIỆN THANH TIẾN TRÌNH
    for i, (src, trg) in enumerate(tqdm(iterator, desc="Training")):

        src = src.to(DEVICE)
        trg = trg.to(DEVICE)

        optimizer.zero_grad()

        output = model(src, trg)

        # output = [trg_len, batch_size, trg_vocab_size]
        output_dim = output.shape[-1]

        # output = [(trg_len - 1) * batch_size, output_dim]
        output = output[1:].view(-1, output_dim)

        # trg = [(trg_len - 1) * batch_size]
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    # THÊM tqdm VÀO CẢ HÀM EVALUATE NỮA
    with torch.no_grad():
        for i, (src, trg) in enumerate(tqdm(iterator, desc="Evaluating")):

            src = src.to(DEVICE)
            trg = trg.to(DEVICE)

            output = model(src, trg, 0) # Teacher forcing = 0

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

# --- VÒNG LẶP HUẤN LUYỆN ---
N_EPOCHS = 15
CLIP = 1
SAVE_FILE = 'tut3-model.pt' # File cho Bài 3
best_valid_loss = float('inf')

print(" Bắt đầu huấn luyện...")
for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, dev_iterator, criterion)

    end_time = time.time()
    epoch_mins = int((end_time - start_time) / 60)
    epoch_secs = int((end_time - start_time) % 60)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), SAVE_FILE)
        print(f" Lưu model mới (Loss: {valid_loss:.3f})")

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Số lượng tham số: 11,656,795
 Bắt đầu huấn luyện...


Training:   0%|          | 0/313 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

 Lưu model mới (Loss: 6.379)
Epoch: 01 | Time: 3m 15s
	Train Loss: 6.282 | Train PPL: 534.809
	 Val. Loss: 6.379 |  Val. PPL: 589.441


Training:   0%|          | 0/313 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

 Lưu model mới (Loss: 5.676)
Epoch: 14 | Time: 2m 42s
	Train Loss: 4.283 | Train PPL:  72.493
	 Val. Loss: 5.676 |  Val. PPL: 291.739


Training:   0%|          | 0/313 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 15 | Time: 2m 45s
	Train Loss: 4.194 | Train PPL:  66.286
	 Val. Loss: 5.702 |  Val. PPL: 299.538


In [16]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8d06c0ff822bb68130ef402429b8b9ec14918ab64e61c87e6db09c011238c886
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [17]:
from rouge_score import rouge_scorer


# Tải mô hình tốt nhất
model.load_state_dict(torch.load('tut3-model.pt'))

def translate_sentence_luong_feeding(sentence, src_vocab, trg_vocab, model, device, max_len = 50):
    model.eval()
    tokens = tokenize_en(sentence)
    src_indexes = src_vocab.lookup_indices(tokens)
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)

    hidden = hidden.unsqueeze(0).repeat(3, 1, 1)
    cell = torch.zeros(3, 1, model.decoder.rnn.hidden_size).to(device)

    # Init input feeding vector
    attentional_hidden = torch.zeros(1, model.decoder.rnn.hidden_size).to(device)

    trg_indexes = [trg_vocab['<sos>']]

    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
        with torch.no_grad():
            # Truyền input feeding
            output, hidden, cell, attentional_hidden = model.decoder(trg_tensor, hidden, encoder_outputs, cell, attentional_hidden)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)
        if pred_token == trg_vocab['<eos>']: break

    trg_tokens = trg_vocab.lookup_tokens(trg_indexes)
    return trg_tokens[1:]

def calculate_rouge_l(model, iterator, src_vocab, trg_vocab, device):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    total_rouge_l = 0
    count = 0

    # Load model tốt nhất (tut3-model.pt hoặc tut3-input-feeding.pt tùy bạn đặt tên)
    model.load_state_dict(torch.load('tut3-model.pt'))

    for src_batch, trg_batch in tqdm(iterator, desc="Calculating ROUGE-L"):
        trg_batch = trg_batch.transpose(0, 1)
        for i in range(trg_batch.shape[0]):
            src_tensor = src_batch[:, i].unsqueeze(1)
            src_indices = src_tensor.squeeze(1).tolist()

            # Lọc token rác
            src_tokens_raw = src_vocab.lookup_tokens(src_indices)
            src_tokens = [t for t in src_tokens_raw if t not in ['<pad>', '<unk>']]
            src_sentence = ' '.join(src_tokens)

            # --- SỬA TẠI ĐÂY: Gọi đúng hàm translate_sentence_luong_feeding ---
            hyp_tokens = translate_sentence_luong_feeding(src_sentence, src_vocab, trg_vocab, model, device)

            hypothesis = ' '.join([t for t in hyp_tokens if t not in ['<unk>', '<eos>', '<pad>']]).strip()

            ref_tokens_raw = trg_vocab.lookup_tokens(trg_batch[i].tolist())
            reference = ' '.join([t for t in ref_tokens_raw if t not in ['<sos>', '<eos>', '<pad>', '<unk>']]).strip()

            if not reference or not hypothesis: continue

            score = scorer.score(reference, hypothesis)
            total_rouge_l += score['rougeL'].fmeasure
            count += 1

    return total_rouge_l / count if count > 0 else 0

# Tính điểm lại
score = calculate_rouge_l(model, test_iterator, vocab_src, vocab_trg, DEVICE)
print(f"ROUGE-L Score: {score:.4f}")

Calculating ROUGE-L:   0%|          | 0/32 [00:00<?, ?it/s]

ROUGE-L Score: 0.3849
